# PROJECT TITLE: INDIAN START-UP FUNDING DATA ANALYSIS

### Project Description/Scenario






This project seeks to gain insight into the fundings received by start-ups companies in India between 2018 and 2021. And advice a team trying to venture into the Indian start-up ecosystem, by proposing the best course of action. This would be done by developing a unique story from this dataset, stating and testing a hypothesis, asking questions, perform analysis and share insights with relevant visualisations.

#### Week 1



###### TASK

Your task for this week is to have a clear understanding of the
project, derive hypothesis and ask questions that will help you
achieve the objectives of the project.


- Title and description of project
- Clearly stated hypothesis
- At least 5 questions

#### Week 2


- Exploratory data analysis of the data provided
- Issues with the data
- How you are going to handle each issue identified

### Week 3

- Clean Data and prepare it for analysis
- Answer Bussines Questions

### BUSINESS UNDERSTANDING


 
Start-up funding plays a crucial role providing essential capital to nurture new ventures that drive economic growth and technological advancement.Indian startups ecosystem span across various sectors and domains, such as e-commerce, fintech, edtech, healthtech, and agritech. 


This project aims to equip the team with the knowledge and strategic insights on identifying the most promising sectors, cities, funding trends, and key players necessary to make informed decisions and successfully engage with the dynamic and rapidly evolving Indian start-up landscape.

### Data Understanding



 
The datasets contains information about startup funding from 2018 to 2021. It includes various attributes such as the company’s name, sector, funding amount, stage, investor details, and location.
 
The key attributes in the dataset include:
 
**Company/Brand**: Name of the company/start-up
 
**Founded**: Year start-up was founded
 
**Sector**: Sector of service
 
**What it does**: Description about Company
 
**Founders**: Founders of the Company
 
**Investor**: Investors
 
**Amount($)**: Raised fund
 
**Stage**: Round of funding reached
 
**Headquarters**: Location of   the starp_company


**Hypothesis**
 
Null Hypothesis (H0): Funding to start-ups is centralized around specific locations and sectors.
 
Alternative Hypothesis (H1): Funding to start-ups is spread across different locations and sectors.
 

 Hypothesis : Start-ups in the technology sector have a higher success rate compared to other sectors.
 H₀: Start-ups in the technology sector do not have a higher success rate compared to start-ups in other sectors.
 
 
**RESEARCH QUESTIONS**
 
1.How has funding to startups changed over the period of time?/What is the trend of funding amounts over the years?

2.What sectors are attracting the most investment in the Indian start-up ecosystem?

3.Which cities are the major hubs for start-ups in India?
 
4.Which headquarter is the most preferred startup location?
 
5.Which sectors are most favoured by investors?
6.Who are the key investors in the Indian start-up ecosystem?

7.What is the success rate of start-ups in different sectors?
 
8.What are the most common funding stages among indian startups?

 ### Importing Liabries and Uploading Datasets

In [1]:
from dotenv import dotenv_values  # Import the dotenv_values function from the dotenv package
import pandas as pd
import numpy as np
import warnings 
import pyodbc
import matplotlib
warnings.filterwarnings("ignore")

In [2]:
environment_variables = dotenv_values('.env')
# we are getting values from our dotenv files to extract the database,server,login and password.

database= environment_variables.get("database")
server=environment_variables.get("server")
username= environment_variables.get("username")
password= environment_variables.get("password")
#The connection string below uses the sql driver to connect the server using the credentials provided
connection_string =f"DRIVER={{SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}"


In [3]:
print( connection_string)

DRIVER={SQL Server};SERVER=dap-projects-database.database.windows.net;DATABASE=dapDB;UID=LP1_learner;PWD=Hyp0th3s!$T3$t!ng


In [4]:
connection= pyodbc.connect(connection_string)
# We now connect to the sql database and find out the tables in this database
db_query ='''SELECT * 
FROM INFORMATION_SCHEMA.TABLES
WHERE TABLE_TYPE = 'BASE TABLE' '''

In [5]:
# Extracting Dataset
data = pd.read_sql(db_query, connection)
data



,TABLE_CATALOG,TABLE_SCHEMA,TABLE_NAME,TABLE_TYPE
0,dapDB,dbo,LP1_startup_funding2021,BASE TABLE
1,dapDB,dbo,LP1_startup_funding2020,BASE TABLE


In [6]:
query= "Select * from dbo.LP1_startup_funding2021 "

query2= "Select * from dbo.LP1_startup_funding2020 "


In [7]:
# Loading dataset

data_2021 = pd.read_sql(query, connection)
data_2020 = pd.read_sql(query2, connection)
data_2019 = pd.read_csv(r"C:\Users\Josephine\Downloads\startup_funding2019.csv")
data_2018 = pd.read_csv(r'C:\Users\Josephine\Downloads\startup_funding2018.csv')



# EDA

#### Exploring the 2018 Dataset

In [8]:
data_2018.head(3)

,Company Name,Industry,Round/Series,Amount,Location,About Company
0,TheCollegeFever,"Brand Marketing, Event Promotion, Marketing, S...",Seed,250000,"Bangalore, Karnataka, India","TheCollegeFever is a hub for fun, fiesta and f..."
1,Happy Cow Dairy,"Agriculture, Farming",Seed,"₹40,000,000","Mumbai, Maharashtra, India",A startup which aggregates milk from dairy far...
2,MyLoanCare,"Credit, Financial Services, Lending, Marketplace",Series A,"₹65,000,000","Gurgaon, Haryana, India",Leading Online Loans Marketplace in India


In [9]:
data_2018.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 526 entries, 0 to 525
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Company Name   526 non-null    object
 1   Industry       526 non-null    object
 2   Round/Series   526 non-null    object
 3   Amount         526 non-null    object
 4   Location       526 non-null    object
 5   About Company  526 non-null    object
dtypes: object(6)
memory usage: 24.8+ KB


In [10]:
data_2018.shape
print("The 2018 Funding data contains " ,data_2018.shape[0], "rows and " ,data_2018.shape[1], "columns")

The 2018 Funding data contains  526 rows and  6 columns


In [11]:
#Checking a brief description of the 2018 dataset

data_2018.describe(include="all").transpose()

,count,unique,top,freq
Company Name,526,525,TheCollegeFever,2
Industry,526,405,—,30
Round/Series,526,21,Seed,280
Amount,526,198,—,148
Location,526,50,"Bangalore, Karnataka, India",102
About Company,526,524,"TheCollegeFever is a hub for fun, fiesta and f...",2


1. - There is 1 duplicate in the Company Name coulumn as "TheCollegeFever" . It also needs some spacing.
2. There are 30 and 148 "-" in Industry and Amount Columns respectively. Would drop them and replace it with unkown...could change
3. Location column is not consistent with the rest of the dataset. It contained City, State, and Country.It would be replaced with city in order to correspond with the other datasets.
4. There are locations that contain India, Asia. This will be replaced with the most recurring city.
5. The 'amount' column is a string instead of float or integer




In [12]:
# Checking for NaN values in the dataset

data_2018.isna().sum()

Company Name     0
Industry         0
Round/Series     0
Amount           0
Location         0
About Company    0
dtype: int64

In [13]:
# Checking for duplicates
data_2018.duplicated().sum()


1

In [14]:
# Investigating the duplicated entry
data_2018[data_2018["Company Name"]=="TheCollegeFever"]

,Company Name,Industry,Round/Series,Amount,Location,About Company
0,TheCollegeFever,"Brand Marketing, Event Promotion, Marketing, S...",Seed,250000,"Bangalore, Karnataka, India","TheCollegeFever is a hub for fun, fiesta and f..."
348,TheCollegeFever,"Brand Marketing, Event Promotion, Marketing, S...",Seed,250000,"Bangalore, Karnataka, India","TheCollegeFever is a hub for fun, fiesta and f..."


- Deep Dive into the "Amount"  Column

In [15]:
data_2018['Amount'].head(20)

# The amount column contains symbols; '$', '₹',',' and '-'
# The symbols would be extracted to attain the int datatype instead of object.
# All Rupees would be converted to Dollars
# The '-' symbol will be replaced with "Unknown" because we assume the money wasnt given.
# Monies without currency signs would be taken as $ 
# Amount column would be renamed as ['Amount($)]

0           250000
1      ₹40,000,000
2      ₹65,000,000
3          2000000
4                —
5          1600000
6      ₹16,000,000
7      ₹50,000,000
8                —
9                —
10               —
11               —
12               —
13               —
14               —
15    ₹100,000,000
16          150000
17         1100000
18               —
19        ₹500,000
Name: Amount, dtype: object

In [16]:
# Further cross-checking of selected columns

# pd.set_option('display.max_rows', None)
data_2018[["Company Name", "Industry", "Round/Series", "Location"]]
# Index 12,42,59,199 had india and asia

,Company Name,Industry,Round/Series,Location
0,TheCollegeFever,"Brand Marketing, Event Promotion, Marketing, S...",Seed,"Bangalore, Karnataka, India"
1,Happy Cow Dairy,"Agriculture, Farming",Seed,"Mumbai, Maharashtra, India"
2,MyLoanCare,"Credit, Financial Services, Lending, Marketplace",Series A,"Gurgaon, Haryana, India"
3,PayMe India,"Financial Services, FinTech",Angel,"Noida, Uttar Pradesh, India"
4,Eunimart,"E-Commerce Platforms, Retail, SaaS",Seed,"Hyderabad, Andhra Pradesh, India"
...,...,...,...,...
521,Udaan,"B2B, Business Development, Internet, Marketplace",Series C,"Bangalore, Karnataka, India"
522,Happyeasygo Group,"Tourism, Travel",Series A,"Haryana, Haryana, India"
523,Mombay,"Food and Beverage, Food Delivery, Internet",Seed,"Mumbai, Maharashtra, India"
524,Droni Tech,Information Technology,Seed,"Mumbai, Maharashtra, India"


In [17]:
# Looking Into Round/Series

pd.set_option('display.max_rows', None)

data_2018['Round/Series'].value_counts()

# Funding type with the highest count was 'Seed'
# Column had a google document link 


Round/Series
Seed                                                                                                       280
Series A                                                                                                    73
Angel                                                                                                       37
Venture - Series Unknown                                                                                    37
Series B                                                                                                    20
Series C                                                                                                    16
Debt Financing                                                                                              13
Private Equity                                                                                              10
Corporate Round                                                                                    

#### Exploring the 2019 Dataset

In [18]:
data_2019.head(10)

,Company/Brand,Founded,HeadQuarter,Sector,What it does,Founders,Investor,Amount($),Stage
0,Bombay Shaving,NaN,NaN,Ecommerce,Provides a range of male grooming products,Shantanu Deshpande,Sixth Sense Ventures,"$6,300,000",NaN
1,Ruangguru,2014.0,Mumbai,Edtech,A learning platform that provides topic-based ...,"Adamas Belva Syah Devara, Iman Usman.",General Atlantic,"$150,000,000",Series C
2,Eduisfun,NaN,Mumbai,Edtech,It aims to make learning fun via games.,Jatin Solanki,"Deepak Parekh, Amitabh Bachchan, Piyush Pandey","$28,000,000",Fresh funding
3,HomeLane,2014.0,Chennai,Interior design,Provides interior designing solutions,"Srikanth Iyer, Rama Harinath","Evolvence India Fund (EIF), Pidilite Group, FJ...","$30,000,000",Series D
4,Nu Genes,2004.0,Telangana,AgriTech,"It is a seed company engaged in production, pr...",Narayana Reddy Punyala,Innovation in Food and Agriculture (IFA),"$6,000,000",NaN
5,FlytBase,NaN,Pune,Technology,A drone automation platform,Nitin Gupta,Undisclosed,Undisclosed,NaN
6,Finly,NaN,Bangalore,SaaS,It builds software products that makes work si...,"Vivek AG, Veekshith C Rai","Social Capital, AngelList India, Gemba Capital...",Undisclosed,NaN
7,Kratikal,2013.0,Noida,Technology,It is a product-based cybersecurity solutions ...,"Pavan Kushwaha, Paratosh Bansal, Dip Jung Thapa","Gilda VC, Art Venture, Rajeev Chitrabhanu.","$1,000,000",Pre series A
8,Quantiphi,NaN,NaN,AI & Tech,It is an AI and big data services company prov...,Renuka Ramnath,Multiples Alternate Asset Management,"$20,000,000",Series A
9,Lenskart,2010.0,Delhi,E-commerce,It is a eyewear company,"Peyush Bansal, Amit Chaudhary, Sumeet Kapahi",SoftBank,"$275,000,000",Series G


In [19]:
data_2019.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89 entries, 0 to 88
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Company/Brand  89 non-null     object 
 1   Founded        60 non-null     float64
 2   HeadQuarter    70 non-null     object 
 3   Sector         84 non-null     object 
 4   What it does   89 non-null     object 
 5   Founders       86 non-null     object 
 6   Investor       89 non-null     object 
 7   Amount($)      89 non-null     object 
 8   Stage          43 non-null     object 
dtypes: float64(1), object(8)
memory usage: 6.4+ KB


In [20]:
data_2019.shape
print("The 2019 Funding data contains " ,data_2019.shape[0], "rows and " ,data_2019.shape[1], "columns")

The 2019 Funding data contains  89 rows and  9 columns


In [21]:
data_2019.describe(include="all").transpose()

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
Company/Brand,89,87,Kratikal,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Founded,60.0,NaN,NaN,NaN,2014.533333,2.937003,2004.0,2013.0,2015.0,2016.25,2019.0
HeadQuarter,70,17,Bangalore,21,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Sector,84,52,Edtech,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN
What it does,89,88,Online meat shop,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Founders,86,85,"Vivek Gupta, Abhay Hanjura",2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Investor,89,86,Undisclosed,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Amount($),89,50,Undisclosed,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Stage,43,15,Series A,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
# Checking for the number of NaN values

data_2019.isna().sum()
# There are NaN values in founded, Headquarters, Sector, Founders and Stage 

Company/Brand     0
Founded          29
HeadQuarter      19
Sector            5
What it does      0
Founders          3
Investor          0
Amount($)         0
Stage            46
dtype: int64

In [23]:
# Checking for duplicated values

data_2019.duplicated().sum()
# There are no duplicates found

0

In [24]:
# Checking selected coulmns for errors
pd.set_option('display.max_rows', None)
data_2019[["Company/Brand", "Sector", "Investor", "Stage"]]

,Company/Brand,Sector,Investor,Stage
0,Bombay Shaving,Ecommerce,Sixth Sense Ventures,NaN
1,Ruangguru,Edtech,General Atlantic,Series C
2,Eduisfun,Edtech,"Deepak Parekh, Amitabh Bachchan, Piyush Pandey",Fresh funding
3,HomeLane,Interior design,"Evolvence India Fund (EIF), Pidilite Group, FJ...",Series D
4,Nu Genes,AgriTech,Innovation in Food and Agriculture (IFA),NaN
5,FlytBase,Technology,Undisclosed,NaN
6,Finly,SaaS,"Social Capital, AngelList India, Gemba Capital...",NaN
7,Kratikal,Technology,"Gilda VC, Art Venture, Rajeev Chitrabhanu.",Pre series A
8,Quantiphi,AI & Tech,Multiples Alternate Asset Management,Series A
9,Lenskart,E-commerce,SoftBank,Series G


### Exporing the 2020 dataset

In [25]:
data_2020.head(20)

,Company_Brand,Founded,HeadQuarter,Sector,What_it_does,Founders,Investor,Amount,Stage,column10
0,Aqgromalin,2019.0,Chennai,AgriTech,Cultivating Ideas for Profit,"Prasanna Manogaran, Bharani C L",Angel investors,200000.0,None,None
1,Krayonnz,2019.0,Bangalore,EdTech,An academy-guardian-scholar centric ecosystem ...,"Saurabh Dixit, Gurudutt Upadhyay",GSF Accelerator,100000.0,Pre-seed,None
2,PadCare Labs,2018.0,Pune,Hygiene management,Converting bio-hazardous waste to harmless waste,Ajinkya Dhariya,Venture Center,NaN,Pre-seed,None
3,NCOME,2020.0,New Delhi,Escrow,Escrow-as-a-service platform,Ritesh Tiwari,"Venture Catalysts, PointOne Capital",400000.0,None,None
4,Gramophone,2016.0,Indore,AgriTech,Gramophone is an AgTech platform enabling acce...,"Ashish Rajan Singh, Harshit Gupta, Nishant Mah...","Siana Capital Management, Info Edge",340000.0,None,None
5,qZense,2019.0,Bangalore,AgriTech,qZense Labs is building the next-generation Io...,"Rubal Chib, Dr Srishti Batra","Venture Catalysts, 9Unicorns Accelerator Fund",600000.0,Seed,None
6,MyClassboard,2008.0,Hyderabad,EdTech,MyClassboard is a full-fledged School / Colleg...,Ajay Sakhamuri,ICICI Bank.,600000.0,Pre-series A,None
7,Metvy,2018.0,Gurgaon,Networking platform,AI driven networking platform for individuals ...,Shawrya Mehrotra,HostelFund,NaN,Pre-series,None
8,Rupeek,2015.0,Bangalore,FinTech,Rupeek is an online lending platform that spec...,"Amar Prabhu, Ashwin Soni, Sumit Maniyar","KB Investment, Bertelsmann India Investments",45000000.0,Series C,None
9,Gig India,2017.0,Pune,Crowdsourcing,GigIndia is a marketplace that provides on-dem...,"Aditya Shirole, Sahil Sharma","Shantanu Deshpande, Subramaniam Ramadorai",1000000.0,Pre-series A,None


In [26]:
data_2020.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1055 entries, 0 to 1054
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Company_Brand  1055 non-null   object 
 1   Founded        842 non-null    float64
 2   HeadQuarter    961 non-null    object 
 3   Sector         1042 non-null   object 
 4   What_it_does   1055 non-null   object 
 5   Founders       1043 non-null   object 
 6   Investor       1017 non-null   object 
 7   Amount         801 non-null    float64
 8   Stage          591 non-null    object 
 9   column10       2 non-null      object 
dtypes: float64(2), object(8)
memory usage: 82.6+ KB


In [27]:
data_2020.shape
print("The 2020 Funding data contains " ,data_2020.shape[0], "rows and " ,data_2020.shape[1], "columns")

The 2020 Funding data contains  1055 rows and  10 columns


In [28]:
# A brief statistical analysis of the columns
data_2019.describe(include="all").transpose()

# We can note that theres a duplicate at the company and founders, What it does column which correspoond

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
Company/Brand,89,87,Kratikal,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Founded,60.0,NaN,NaN,NaN,2014.533333,2.937003,2004.0,2013.0,2015.0,2016.25,2019.0
HeadQuarter,70,17,Bangalore,21,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Sector,84,52,Edtech,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN
What it does,89,88,Online meat shop,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Founders,86,85,"Vivek Gupta, Abhay Hanjura",2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Investor,89,86,Undisclosed,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Amount($),89,50,Undisclosed,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Stage,43,15,Series A,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
# Checkng for NaN values
data_2020.isna().sum()

Company_Brand       0
Founded           213
HeadQuarter        94
Sector             13
What_it_does        0
Founders           12
Investor           38
Amount            254
Stage             464
column10         1053
dtype: int64

In [30]:
# Checking for duplicated values

data_2020.duplicated().sum()
# Good, This confirms our duplicates at the description section.

3

In [31]:
# Lets find out whats in column 10
pd.set_option('display.max_rows', None)
data_2020['column10']

# There are only two objects in column10, will be dropped later in data cleaning.

0             None
1             None
2             None
3             None
4             None
5             None
6             None
7             None
8             None
9             None
10            None
11            None
12            None
13            None
14            None
15            None
16            None
17            None
18            None
19            None
20            None
21            None
22            None
23            None
24            None
25            None
26            None
27            None
28            None
29            None
30            None
31            None
32            None
33            None
34            None
35            None
36            None
37            None
38            None
39            None
40            None
41            None
42            None
43            None
44            None
45            None
46            None
47            None
48            None
49            None
50            None
51            None
52          

In [32]:
# Dropping The last Column because there are only 2 values in it
data_2020['column10']
data_2020.drop(['column10'],axis=1,inplace=True)

In [33]:
# Looking into the duplicated values


### Exploring 2021 dataset


In [34]:
data_2021.tail(10)

,Company_Brand,Founded,HeadQuarter,Sector,What_it_does,Founders,Investor,Amount,Stage
1199,Proeon,2018.0,Pune,Food Production,Innovating plant protein ingredients with supe...,"Ashish Korde, Kevin Parekh","Shaival Desai, Flowstate Ventures",$2000000,Seed
1200,InfyU Labs,2019.0,Gandhinagar,AgriTech,InfyU Labs is a team of dedicated professional...,"Amit Srivastava, Ankit Chauhan",IAN,$200000,Seed
1201,TechEagle,2015.0,Gurugram,Aviation & Aerospace,"Safe, secure & reliable On-Demand Drone delive...",Vikram Singh Meena,India Accelerator,$500000,Seed
1202,Voxelgrids,2017.0,Bangalore,Deeptech,Voxelgrids is an Magnetic Resonance Imaging te...,Arjun Arunachalam,Zoho,$5000000,None
1203,Cogos Technologies,2016.0,Bangalore,Logistics & Supply Chain,A smart-tech-enabled platform offering a one-s...,Prasad Sreeram,Transworld Group,$2000000,Pre-series A
1204,Gigforce,2019.0,Gurugram,Staffing & Recruiting,A gig/on-demand staffing company.,"Chirag Mittal, Anirudh Syal",Endiya Partners,$3000000,Pre-series A
1205,Vahdam,2015.0,New Delhi,Food & Beverages,VAHDAM is among the world’s first vertically i...,Bala Sarda,IIFL AMC,$20000000,Series D
1206,Leap Finance,2019.0,Bangalore,Financial Services,International education loans for high potenti...,"Arnav Kumar, Vaibhav Singh",Owl Ventures,$55000000,Series C
1207,CollegeDekho,2015.0,Gurugram,EdTech,"Collegedekho.com is Student’s Partner, Friend ...",Ruchir Arora,"Winter Capital, ETS, Man Capital",$26000000,Series B
1208,WeRize,2019.0,Bangalore,Financial Services,India’s first socially distributed full stack ...,"Vishal Chopra, Himanshu Gupta","3one4 Capital, Kalaari Capital",$8000000,Series A


In [35]:
data_2021.shape
print("The 2021 Funding data contains " ,data_2021.shape[0], "rows and " ,data_2021.shape[1], "columns")

The 2021 Funding data contains  1209 rows and  9 columns


In [36]:
data_2021.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1209 entries, 0 to 1208
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Company_Brand  1209 non-null   object 
 1   Founded        1208 non-null   float64
 2   HeadQuarter    1208 non-null   object 
 3   Sector         1209 non-null   object 
 4   What_it_does   1209 non-null   object 
 5   Founders       1205 non-null   object 
 6   Investor       1147 non-null   object 
 7   Amount         1206 non-null   object 
 8   Stage          781 non-null    object 
dtypes: float64(1), object(8)
memory usage: 85.1+ KB


In [37]:
# A brief description of the data

data_2021.describe(include="all").transpose()
# Check The BharatPe

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
Company_Brand,1209,1033,BharatPe,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Founded,1208.0,NaN,NaN,NaN,2016.655629,4.517364,1963.0,2015.0,2018.0,2020.0,2021.0
HeadQuarter,1208,70,Bangalore,426,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Sector,1209,254,FinTech,122,NaN,NaN,NaN,NaN,NaN,NaN,NaN
What_it_does,1209,1143,BharatPe develops a QR code-based payment app ...,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Founders,1205,1095,"Ashneer Grover, Shashvat Nakrani",7,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Investor,1147,937,Inflection Point Ventures,24,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Amount,1206,278,$Undisclosed,73,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Stage,781,31,Seed,246,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [38]:
# Checking for duplicated values

data_2021.duplicated().sum()

19

In [39]:
# Checking for null values
data_2021.isna().sum()
# there are null values in Founded, "Sector", "Investor","Founders" "Amount","Stage" &/ "HeadQuarter

Company_Brand      0
Founded            1
HeadQuarter        1
Sector             0
What_it_does       0
Founders           4
Investor          62
Amount             3
Stage            428
dtype: int64

In [40]:
# pd.set_option('display.max_rows', None)
data_2021[["Company_Brand", "Sector", "Investor","Founded" ,"Stage", "HeadQuarter"]]

,Company_Brand,Sector,Investor,Founded,Stage,HeadQuarter
0,Unbox Robotics,AI startup,"BEENEXT, Entrepreneur First",2019.0,Pre-series A,Bangalore
1,upGrad,EdTech,"Unilazer Ventures, IIFL Asset Management",2015.0,None,Mumbai
2,Lead School,EdTech,"GSV Ventures, Westbridge Capital",2012.0,Series D,Mumbai
3,Bizongo,B2B E-commerce,"CDC Group, IDG Capital",2015.0,Series C,Mumbai
4,FypMoney,FinTech,"Liberatha Kallat, Mukesh Yadav, Dinesh Nagpal",2021.0,Seed,Gurugram
5,Urban Company,Home services,Vy Capital,2014.0,None,New Delhi
6,Comofi Medtech,HealthTech,"CIIE.CO, KIIT-TBI",2018.0,None,Bangalore
7,Qube Health,HealthTech,Inflection Point Ventures,2016.0,Pre-series A,Mumbai
8,Vitra.ai,Tech Startup,Inflexor Ventures,2020.0,None,Bangalore
9,Taikee,E-commerce,None,2010.0,None,Mumbai


In [41]:
print ("There are ",len(data_2021[data_2021.duplicated()]) ,"duplicated values in the dataset")

There are  19 duplicated values in the dataset


## Issues identified during Exploratory Data Analysis (EDA) stage
#### Data_2018
1. Amount has datatype as object. It should be a float data type
2. There are different currencies in the Amount column. All currencies are supposed to be in dollars. 
 All commas, dashes attached to the Amounts should be removed. 
3. There are no Non-Avalaible Values(NaN) in the dataset
4. Company names mixed with website. Cell 31 =' Ideal insurance.com plus others...We may decide to leave it as it is.
5. There is a google document link in the Rounds/Series column
6. The "About" column has no influence on our analysis so hence must be removed.(Lets decide)
7. There is 1 duplicated information in the dataset(Company Name = TheCollegeFever)
8. Location column is not consistent with the rest of the dataset. It contained city, State, and Country.
9. There are locations that contain India, Asia. This will be replaced with the most recurring city.
10. Location would be changed to Headquarters to match with the rest of the datasets

#### Data_2019
1. There are lots NaN values*** in the Stage, HeadQuaters and Founded columns
2. There is the currency sign and commas attached to the amounts would be removed and have amount changed as Amount($)
3. The Founded Column is in float. No big issue.
4.  Amount column is in object.would be changed to Integer or Float
5. The Founder columnn is not important to our analysis
6. Company/Brand column must be renamed to Company_Name

#### Data_2020
1. Company/Brand column must be renamed to Company_Name
2. Column10" has no influence on the dataset
3. There is the currency sign,dashes and commas attached to the amounts.
4. Spelling error (>Vikram Sud, row=192) in the investor column
5. There are Headquaters cities that are outside India.eg
6. Duplicated three entries (Byju’s, Nykaa, Krimanshi)
7. Columns with names of cities and state.would maintain just the city

#### Data_2021
1. Company/Brand column would be renamed to Company_Name
2. There is the currency sign and commas attached to the amounts.
4. There are lots of NaN values
5. There are duplicated values (Curefoods, Bewakoofs, FanPlay, Advantage Club, Ruptok, Trinker, Zorro, Ultraviollette, Nephroplus, Unremot, Fansanywhere,Pingolearn, Spy, Enmovil, ASQI Advisers, Insurance Samadhan, Evenflow Brands, MasterChow, Fullife healthcare)
6. There are instances where values have been recorded under the wrong 

#### General Additions
1. Funding_year would be added to the datasets to know which year these companies recieved funding
2. 'Founded','Founders' , 'About company' or 'what it does' Columns would be dropped 
3.  Company/Brand column would be renamed to Company_Name
4. Some columns would have their Datatype changed into the required format. 
5. Add Investor column to 2018 dataset and fill with 'Unknown'
 

## Data Cleaning And Preparation

1. All identified errors or issues in the dataset will be corrected in the cleaning process.
2. The datasets will be concactenanted into a single dataset

In [42]:
# Add the funding_year to the dataset. 
data_2020.insert (7,'funding_year',2020)
data_2021.insert (7,'funding_year',2021)
data_2019.insert (7,'funding_year',2019)
data_2018.insert (4,'funding_year',2018)

In [43]:
# Changing the Company Brand to Company Name 
# Rename the 'Company_Brand' column to 'Company_Name'
data_2021.rename(columns={'Company_Brand': 'Company_Name'}, inplace=True)
data_2020.rename(columns={'Company_Brand': 'Company_Name'}, inplace=True)
data_2019.rename(columns={'Company/Brand': 'Company_Name'}, inplace=True)
data_2018.rename(columns={'Company Name' : 'Company_Name'}, inplace=True)

In [44]:
data_2021.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1209 entries, 0 to 1208
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Company_Name  1209 non-null   object 
 1   Founded       1208 non-null   float64
 2   HeadQuarter   1208 non-null   object 
 3   Sector        1209 non-null   object 
 4   What_it_does  1209 non-null   object 
 5   Founders      1205 non-null   object 
 6   Investor      1147 non-null   object 
 7   funding_year  1209 non-null   int64  
 8   Amount        1206 non-null   object 
 9   Stage         781 non-null    object 
dtypes: float64(1), int64(1), object(8)
memory usage: 94.6+ KB


In [45]:
 # Add investor column to 2018 dataset and fill with NaN
data_2018.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 526 entries, 0 to 525
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Company_Name   526 non-null    object
 1   Industry       526 non-null    object
 2   Round/Series   526 non-null    object
 3   Amount         526 non-null    object
 4   funding_year   526 non-null    int64 
 5   Location       526 non-null    object
 6   About Company  526 non-null    object
dtypes: int64(1), object(6)
memory usage: 28.9+ KB


In [46]:
data_2019.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89 entries, 0 to 88
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Company_Name  89 non-null     object 
 1   Founded       60 non-null     float64
 2   HeadQuarter   70 non-null     object 
 3   Sector        84 non-null     object 
 4   What it does  89 non-null     object 
 5   Founders      86 non-null     object 
 6   Investor      89 non-null     object 
 7   funding_year  89 non-null     int64  
 8   Amount($)     89 non-null     object 
 9   Stage         43 non-null     object 
dtypes: float64(1), int64(1), object(8)
memory usage: 7.1+ KB


##### Renaming Columns

In [47]:
# Rename Round/Series to Stage
data_2018.rename(columns={'Round/Series': 'Stage'}, inplace=True)

# Renaming Location To Headquarters.
data_2018.rename(columns={'Location': 'HeadQuarter'}, inplace=True)

# Rename the 'About Company' column to 'What it does'
data_2018.rename(columns={'About Company': 'What_it_does'}, inplace=True)
data_2019.rename(columns={'What it does': 'What_it_does'},inplace=True)

# Renaming 'Industry' to 'Sector'
data_2018.rename(columns={'Industry': 'Sector'}, inplace=True)

# Rename the 'Amount($)'column to 'Amount' to match the other data
data_2019.rename(columns={'Amount($)': 'Amount'},inplace=True)



### Droping Columns to facilitate Merging 

In [48]:

columns_to_drop = ["Founded", "Founders"]
data_2021 = data_2021.drop(columns=columns_to_drop)
data_2020 = data_2020.drop(columns=columns_to_drop)
data_2019 = data_2019.drop(columns=columns_to_drop)

 # PS:Should we drop 'What it does'/About company?
 

### Merging Datasets

In [49]:
data_combined = pd.concat([data_2021, data_2020, data_2019,data_2018], ignore_index=True)
data_combined.head(10)

,Company_Name,HeadQuarter,Sector,What_it_does,Investor,funding_year,Amount,Stage
0,Unbox Robotics,Bangalore,AI startup,Unbox Robotics builds on-demand AI-driven ware...,"BEENEXT, Entrepreneur First",2021,"$1,200,000",Pre-series A
1,upGrad,Mumbai,EdTech,UpGrad is an online higher education platform.,"Unilazer Ventures, IIFL Asset Management",2021,"$120,000,000",None
2,Lead School,Mumbai,EdTech,LEAD School offers technology based school tra...,"GSV Ventures, Westbridge Capital",2021,"$30,000,000",Series D
3,Bizongo,Mumbai,B2B E-commerce,Bizongo is a business-to-business online marke...,"CDC Group, IDG Capital",2021,"$51,000,000",Series C
4,FypMoney,Gurugram,FinTech,"FypMoney is Digital NEO Bank for Teenagers, em...","Liberatha Kallat, Mukesh Yadav, Dinesh Nagpal",2021,"$2,000,000",Seed
5,Urban Company,New Delhi,Home services,Urban Company (Formerly UrbanClap) is a home a...,Vy Capital,2021,"$188,000,000",None
6,Comofi Medtech,Bangalore,HealthTech,Comofi Medtech is a healthcare robotics startup.,"CIIE.CO, KIIT-TBI",2021,"$200,000",None
7,Qube Health,Mumbai,HealthTech,India's Most Respected Workplace Healthcare Ma...,Inflection Point Ventures,2021,Undisclosed,Pre-series A
8,Vitra.ai,Bangalore,Tech Startup,Vitra.ai is an AI-based video translation plat...,Inflexor Ventures,2021,Undisclosed,None
9,Taikee,Mumbai,E-commerce,"Taikee is the ISO-certified, B2B e-commerce pl...",None,2021,"$1,000,000",None


In [50]:
data_combined.dtypes

Company_Name    object
HeadQuarter     object
Sector          object
What_it_does    object
Investor        object
funding_year     int64
Amount          object
Stage           object
dtype: object

#### Cleaning The HeadQuarter Column


In [51]:
#Cleaning up Headquarters to have the cities only
 
data_combined["HeadQuarter"]=data_combined['HeadQuarter'].str.split(',').str[0]
data_combined['HeadQuarter']

0                               Bangalore
1                                  Mumbai
2                                  Mumbai
3                                  Mumbai
4                                Gurugram
5                               New Delhi
6                               Bangalore
7                                  Mumbai
8                               Bangalore
9                                  Mumbai
10                                 Mumbai
11                              New Delhi
12                              Bangalore
13                              Hyderabad
14                                 Jaipur
15                              Bangalore
16                              New Delhi
17                              Bangalore
18                              New Delhi
19                              Bangalore
20                              Bangalore
21                              Bangalore
22                              New Delhi
23                              Ah

In [52]:
# Replace Bangalore with the right name Bengaluru
data_combined["HeadQuarter"] = data_combined['HeadQuarter'].replace("Bangalore", "Bengaluru")
data_combined["HeadQuarter"] = data_combined['HeadQuarter'].replace("Haryana", "Chandigarh")
data_combined["HeadQuarter"].tail(20)

2859     Bengaluru
2860        Mumbai
2861     New Delhi
2862        Mumbai
2863     Bengaluru
2864        Mumbai
2865        Jaipur
2866        Mumbai
2867     New Delhi
2868          Pune
2869     New Delhi
2870     Bengaluru
2871     Ghaziabad
2872     Bengaluru
2873     Bengaluru
2874     Bengaluru
2875    Chandigarh
2876        Mumbai
2877        Mumbai
2878       Chennai
Name: HeadQuarter, dtype: object

In [53]:
# Dealing with null values.
# Calculate the mode of the 'HeadQuarter' column
most_common_city = data_combined['HeadQuarter'].mode()[0]
most_common_city


'Bengaluru'

In [54]:
# Fill 
data_combined["HeadQuarter"].isnull().sum()

# The total amount of null values in headquarters is 114, should we drop or replace 

114

In [55]:
data_combined = data_combined.drop_duplicates(subset="HeadQuarter", keep="first")
data_combined["HeadQuarter"]
# The plan is to drop duplicates but to keep the early occcurencies
# but wrong code...

0                               Bengaluru
1                                  Mumbai
4                                Gurugram
5                               New Delhi
13                              Hyderabad
14                                 Jaipur
23                              Ahmadabad
24                                Chennai
29                                   None
32                            Small Towns
36                                    Goa
37                              Rajsamand
41                                 Ranchi
43                              Faridabad
57                                Gujarat
67                                   Pune
74                                  Thane
98                         Computer Games
100                                Cochin
129                                 Noida
147                            Chandigarh
212                               Gurgaon
219                              Vadodara
241                      Food & Be

##### Cleaning "Amount" Column

In [56]:
# whats left?... Converting amount to dollars and renaming the column as Amount($)
# Handling the duplicate values and null values

In [57]:

# List of specific indices to replace with NaN 
indices_to_replace = [98, 111, 677, 551, 545, 538, 256, 257, 242,1148]

data_combined.loc[indices_to_replace, 'Amount'] = 'NaN'

# Replace 'Undisclosed', '$Undisclosed', '—', '-', and other non-numeric values with 
data_combined["Amount"] = data_combined["Amount"].replace("Undisclosed", 'Nan')
data_combined["Amount"] = data_combined["Amount"].replace("undisclosed", 'Nan')
data_combined['Amount'] = data_combined['Amount'].replace('$undisclosed', 'NaN')
data_combined['Amount'] = data_combined['Amount'].replace('$Undisclosed', 'NaN')                                             
data_combined['Amount'] = data_combined['Amount'].replace('-', 'NaN')
data_combined['Amount'] = data_combined['Amount'].replace('—', 'NaN')
data_combined[['Amount']].head(9)


KeyError: '[111, 677, 551, 545, 538, 256, 1148] not in index'

In [ ]:
# Changing the Amount data type from object to float or integer by removing the currency signs and other special symbols(,)
# Removing special characters ("," , "$", "-") and Inputing empty rows with NaN. 


data_combined["Amount"].replace(",", "", regex=True, inplace=True)
data_combined["Amount"].replace("$", "", regex=True, inplace=True)
data_combined["Amount"].replace("—", 'NaN', regex=True, inplace=True)
data_combined[['Amount']].fillna(0)
data_combined[['Amount']].head(10)


,Amount
0,1200000
1,120000000
2,30000000
3,51000000
4,2000000
5,188000000
6,200000
7,Nan
8,Nan
9,1000000


In [ ]:
#Extract the symbols into new column "currency"
data_combined['currency'] = data_combined.Amount.str.extract(r'([$₹])')
data_combined[['currency']].head(10)





In [ ]:
# Removing the symbols from the column
data_combined['Amount'] = data_combined['Amount'].str.replace('[$₹—,]', '', regex=True)
data_combined[['Amount']]


# Lots of nan values interferring, should work on it later


#### Cleaning Stage Column



#### Fill NaN values with "Undisclosed"

In [ ]:
data_combined['Stage'] = data_combined['Stage'].fillna("Undisclosed")
data_combined[['Stage']]

In [ ]:
data_combined.isna().sum()

2

### I havent saved as a Csv file because data cleaning not done yet.  Im Cosidering cleaning each data singly and then merging it. Pls comment on this and lets see what we can redo...pls help with the cides wai. Goodnight

In [ ]:
# Display the Amount column for further information
#pd.set_option('display.max_rows', None)

#data_2018[['Amount']].head(5)

# The "₹" and "$" would be extracted from the data and moved into a new column, 'currency'
# The "—" should be removed
# Convert the "	₹" to $" values
# And also handle the null values


In [ ]:
# Convert Amount to a numeric column
#data_2018['Amount'] = pd.to_numeric(data_2018['Amount'])

# Change the Dtype to Integer
#data_2018['Amount'] = data_2018['Amount'].astype('Int64')
#data_2018.dtypes

##### Converting Rupees'₹' to "$"Dollars